In [1]:
import ee
import geemap
import numpy as np

# 初始化Earth Engine
ee.Authenticate()
ee.Initialize(project='socd-liuziyan')

# 加载JAXA ALOS AW3D30数据
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V4_1')
elevation_collection = dataset.select('DSM')

# 获取投影信息并创建镶嵌图像
proj = elevation_collection.first().select(0).projection()
elevation = elevation_collection.mosaic().setDefaultProjection(proj)

# 计算坡度（以度为单位）- 使用重新投影的方法
slope = ee.Terrain.slope(elevation)

# 计算7公里半径内的局部高程变化
# 创建7公里的圆形kernel
kernel = ee.Kernel.circle(radius=7000, units='meters')

# 计算局部高程范围（最大值-最小值）
elevation_max = elevation.reduceNeighborhood(
    reducer=ee.Reducer.max(),
    kernel=kernel
)
elevation_min = elevation.reduceNeighborhood(
    reducer=ee.Reducer.min(),
    kernel=kernel
)
local_elevation_range = elevation_max.subtract(elevation_min)

# 创建纬度掩膜（23°N~19°S之外的区域）
# 生成纬度图像
lat = ee.Image.pixelLonLat().select('latitude')
lat_mask = lat.gt(23).Or(lat.lt(-19))

# 定义山地分类函数
def classify_mountains():
    # 初始化结果图像
    mountain_type = ee.Image.constant(0)
    
    # 类型1：海拔 > 4500m（全球范围）
    type1 = elevation.gt(4500)
    mountain_type = mountain_type.where(type1, 1)
    
    # 类型2：海拔 3500-4500m（全球范围）
    type2 = elevation.gte(3500).And(elevation.lte(4500))
    mountain_type = mountain_type.where(type2, 2)
    
    # 类型3：海拔 2500-3500m（全球范围）
    type3 = elevation.gte(2500).And(elevation.lte(3500))
    mountain_type = mountain_type.where(type3, 3)
    
    # 类型4：海拔 1500-2500m 且坡度≥2°（全球范围）
    type4 = elevation.gte(1500).And(elevation.lte(2500)).And(slope.gte(2))
    mountain_type = mountain_type.where(type4, 4)
    
    # 类型5：海拔 1000-1500m 且（坡度≥5° 或 7km半径内局部高程变化>300m）（全球范围）
    type5_elevation = elevation.gte(1000).And(elevation.lte(1500))
    type5_slope = slope.gte(5)
    type5_relief = local_elevation_range.gt(300)
    type5 = type5_elevation.And(type5_slope.Or(type5_relief))
    mountain_type = mountain_type.where(type5, 5)
    
    # 类型6：海拔 300-1000m 且 7km半径内局部高程变化>300m 且位于纬度23°N~19°S之外
    type6_elevation = elevation.gte(300).And(elevation.lte(1000))
    type6_relief = local_elevation_range.gt(300)
    type6 = type6_elevation.And(type6_relief).And(lat_mask)
    mountain_type = mountain_type.where(type6, 6)
    
    return mountain_type

# 执行山地分类
mountain_classification = classify_mountains()

# 创建掩膜，只保留山地区域（类型1-6）
mountain_mask = mountain_classification.gt(0)
mountain_areas = mountain_classification.updateMask(mountain_mask)

# 创建可视化参数
elevation_vis = {
    'min': 0,
    'max': 5000,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

mountain_vis = {
    'min': 1,
    'max': 6,
    'palette': ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#66bd63']
}

# 创建地图
Map = geemap.Map(center=[0, 0], zoom=2)

# 添加图层
Map.addLayer(elevation, elevation_vis, 'ALOS AW3D30 Elevation')
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['blue', 'yellow', 'red']}, 'Slope (degrees)')
Map.addLayer(mountain_areas, mountain_vis, 'Mountain Classification (UNEP)')

# 添加图例
legend_dict = {
    'Type 1 (>4500m)': '#d73027',
    'Type 2 (3500-4500m)': '#f46d43', 
    'Type 3 (2500-3500m)': '#fdae61',
    'Type 4 (1500-2500m, slope≥2°)': '#fee08b',
    'Type 5 (1000-1500m, conditions)': '#d9ef8b',
    'Type 6 (300-1000m, conditions)': '#66bd63'
}
Map.add_legend(legend_dict=legend_dict, title="Mountain Types (UNEP)")

# 显示地图
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
mountain_areas = mountain_areas.gt(0).toInt8()
# 新建cell：将mountain_areas保存到Asset
# 设置导出参数
asset_id = 'projects/socd-liuziyan/assets/global_mountain_classification_UNEP'  # 请替换为您的用户名

# 导出到Asset
export_task = ee.batch.Export.image.toAsset(
    image=mountain_areas.toInt8(),
    description='Global_Mountain_Classification_UNEP',
    assetId=asset_id,
    scale=1000,  # 30米分辨率
    # region=ee.Geometry.Polygon([[-180, -90], [180, -90], [180, 90], [-180, 90]]),
    maxPixels=1e11,
    # crs='EPSG:4326'
)

# 启动导出任务
export_task.start()

print(f"山地分类结果导出任务已启动")
print(f"Asset ID: {asset_id}")
print("请在Earth Engine Code Editor的Tasks面板中查看导出进度")

# 检查任务状态
import time
print("任务状态检查...")
while export_task.active():
    print(f"任务状态: {export_task.status()['state']}")
    time.sleep(30)  # 每30秒检查一次
    
print(f"最终任务状态: {export_task.status()}")

山地分类结果导出任务已启动
Asset ID: projects/socd-liuziyan/assets/global_mountain_classification_UNEP
请在Earth Engine Code Editor的Tasks面板中查看导出进度
任务状态检查...
任务状态: READY
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
任务状态: RUNNING
最终任务状态: {'state': 'COMPLETED', 'description': 'Global_Mountain_Classification_UNEP', 'priority': 100, 'creation_timestamp_ms': 1754956793003, 'update_timestamp_ms': 1754957357818, 'start_timestamp_ms': 1754956795924, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/socd-liuziyan/assets/global_mountain_classification_UNEP'], 'attempt': 1, 'batch_eecu_usage_seconds': 9530.5791015625, 'id': '6WROI5RAZBLJLONONMIIGK3E', 'name': 'projects/socd-liuziyan/operations/6WROI5RAZBLJLONONMIIGK3E'}


In [ ]:
# 新建cell：使用geemap下载Assets中的影像
import os
import geemap

# 加载Assets中的影像
image_asset_id = "projects/soc-ele/assets/global_mountain_classification_UNEP"
mountain_image = ee.Image(image_asset_id)

# 创建输出目录
output_dir = 'd:/0_phd_papers/2_SOC_Elevation/1_code/downloaded_data'
os.makedirs(output_dir, exist_ok=True)

# 方法1：直接下载整个影像
output_filename = os.path.join(output_dir, 'global_mountain_classification_UNEP.tif')

try:
    geemap.ee_export_image(
        mountain_image,
        filename=output_filename,
        scale=1000,  # 1km分辨率
        region=ee.Geometry.Rectangle([-180, -90, 180, 90])
        # file_per_band=False,
        # timeout=300
    )
    print(f"影像下载完成: {output_filename}")
except Exception as e:
    print(f"下载失败: {e}")




下载失败: ee_export_image() got an unexpected keyword argument 'image'
